In [2]:
# Step 1: Setup Environment
# Install dependencies (if needed)
!pip install ultralytics
!pip install torch torchvision torchaudio

import torch
import os
import shutil
import random
import numpy as np
from sklearn.model_selection import KFold
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:

# Check GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [4]:
!pip install ultralytics opencv-python matplotlib

import torch
import os
import shutil
import random
import numpy as np
import glob
from sklearn.model_selection import KFold
from ultralytics import YOLO
import kagglehub


In [5]:

# Step 2: Download & Load Dataset
path = kagglehub.dataset_download("karanwxlia/underwater-trash-detection")
print("Path to dataset files:", path)

100%|██████████| 213M/213M [00:02<00:00, 75.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/karanwxlia/underwater-trash-detection/versions/1


In [6]:

dataset_path = "/root/.cache/kagglehub/datasets/karanwxlia/underwater-trash-detection/versions/1"
yolo_path = os.path.join(dataset_path, "utd2.v8i.yolov8")


In [7]:

# Define paths for consolidated dataset
all_images_path = "/kaggle/working/all_images"
all_labels_path = "/kaggle/working/all_labels"
os.makedirs(all_images_path, exist_ok=True)
os.makedirs(all_labels_path, exist_ok=True)

In [8]:
# Move images and labels from train, valid, and test to single folders
for split in ["train", "valid", "test"]:
    split_images = os.path.join(yolo_path, split, "images")
    split_labels = os.path.join(yolo_path, split, "labels")

    for img in glob.glob(os.path.join(split_images, "*.jpg")):
        shutil.move(img, all_images_path)
    for lbl in glob.glob(os.path.join(split_labels, "*.txt")):
        shutil.move(lbl, all_labels_path)

print("Merged all images and labels into all_images/ and all_labels/.")


Merged all images and labels into all_images/ and all_labels/.


In [9]:

# Step 3: Prepare 5-Fold Cross Validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)


In [10]:

# Get all images
all_images = sorted(glob.glob(os.path.join(all_images_path, "*.jpg")))
random.shuffle(all_images)

assert len(all_images) > 0, "No images found! Check dataset path."

total_images = len(all_images)
print(f"Total images: {total_images}")

Total images: 9576


In [11]:

# Step 4: Perform Cross Validation
results = []
for fold, (train_idx, val_idx) in enumerate(kf.split(all_images)):
    print(f"\nFold {fold + 1}/{num_folds}...")

    fold_path = f"/kaggle/working/fold_{fold + 1}"
    os.makedirs(fold_path, exist_ok=True)

    fold_train = os.path.join(fold_path, "train")
    fold_val = os.path.join(fold_path, "val")
    os.makedirs(os.path.join(fold_train, "images"), exist_ok=True)
    os.makedirs(os.path.join(fold_train, "labels"), exist_ok=True)
    os.makedirs(os.path.join(fold_val, "images"), exist_ok=True)
    os.makedirs(os.path.join(fold_val, "labels"), exist_ok=True)

    # Move training data
    for idx in train_idx:
        img_path = all_images[idx]
        shutil.copy(img_path, os.path.join(fold_train, "images"))
        lbl_path = img_path.replace("all_images", "all_labels").replace(".jpg", ".txt")
        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, os.path.join(fold_train, "labels"))

    # Move validation data
    for idx in val_idx:
        img_path = all_images[idx]
        shutil.copy(img_path, os.path.join(fold_val, "images"))
        lbl_path = img_path.replace("all_images", "all_labels").replace(".jpg", ".txt")
        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, os.path.join(fold_val, "labels"))



Fold 1/5...

Fold 2/5...

Fold 3/5...

Fold 4/5...

Fold 5/5...


In [12]:

    # Step 5: Create data.yaml for YOLO
    data_yaml = "/content/data.yml"

    # Step 6: Train YOLOv11 on this fold
    model = YOLO("yolo11n.pt")

    results.append(model.train(
        data=data_yaml,
        epochs=25,
        imgsz=640,
        batch=16,
        device=device
    ))

100%|██████████| 5.35M/5.35M [00:00<00:00, 95.3MB/s]


Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/data.yml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, li

100%|██████████| 755k/755k [00:00<00:00, 19.2MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/fold_1/train/labels... 7660 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7660/7660 [00:03<00:00, 2168.72it/s]

...

      25/25      3.12G      1.034     0.6626      1.298         18        640: 100%|██████████| 479/479 [02:14<00:00,  3.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 60/60 [00:15<00:00,  3.76it/s]

                   all       1916       2791      0.905      0.829      0.919      0.686



25 epochs completed in 1.107 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 60/60 [00:19<00:00,  3.00it/s]


                   all       1916       2791      0.905       0.83      0.919      0.686
                   Bio        452        564      0.897      0.867       0.93      0.718
                   Rov        307        391      0.876      0.831      0.913      0.736
                 Trash       1491       1836      0.942      0.792      0.915      0.603
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train


In [13]:
import pandas as pd
results = pd.read_csv("/content/runs/detect/train/results.csv")
results.head()

,epoch,time,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,182.984,1.50096,2.67851,1.62158,0.47374,0.29570,0.33272,0.16035,1.66724,2.45773,1.85676,0.000475,0.000475,0.000475
1,2,346.743,1.58234,2.18220,1.68974,0.44563,0.40744,0.34672,0.16714,1.81194,2.28641,1.91651,0.000914,0.000914,0.000914
2,3,513.197,1.58496,2.02900,1.70288,0.38788,0.36507,0.31717,0.15760,1.82668,2.27073,1.98559,0.001315,0.001315,0.001315
3,4,675.897,1.56608,1.90930,1.67623,0.53966,0.44608,0.46503,0.23609,1.71284,1.78674,1.82872,0.001259,0.001259,0.001259
4,5,838.275,1.50238,1.78364,1.62850,0.53080,0.44621,0.47263,0.25611,1.69607,2.04969,1.87314,0.001203,0.001203,0.001203


In [15]:
import pandas as pd

# Load results CSV
results = pd.read_csv("/content/runs/detect/train/results.csv")

# Extract the last epoch's metrics
final_metrics = results.iloc[-1]  # Get last row

# Print all relevant metrics
print("\nFinal Cross-Validation Results:")
print(f"Epoch: {int(final_metrics['epoch'])}")
print(f"Training Box Loss: {final_metrics['train/box_loss']:.4f}")
print(f"Training Class Loss: {final_metrics['train/cls_loss']:.4f}")
print(f"Training DFL Loss: {final_metrics['train/dfl_loss']:.4f}")
print(f"Validation Box Loss: {final_metrics['val/box_loss']:.4f}")
print(f"Validation Class Loss: {final_metrics['val/cls_loss']:.4f}")
print(f"Validation DFL Loss: {final_metrics['val/dfl_loss']:.4f}")
print(f"Precision: {final_metrics['metrics/precision(B)']:.4f}")
print(f"Recall: {final_metrics['metrics/recall(B)']:.4f}")
print(f"mAP50: {final_metrics['metrics/mAP50(B)']:.4f}")
print(f"mAP50-95: {final_metrics['metrics/mAP50-95(B)']:.4f}")



Final Cross-Validation Results:
Epoch: 25
Training Box Loss: 1.0338
Training Class Loss: 0.6625
Training DFL Loss: 1.2976
Validation Box Loss: 1.0872
Validation Class Loss: 0.6284
Validation DFL Loss: 1.2775
Precision: 0.9054
Recall: 0.8295
mAP50: 0.9192
mAP50-95: 0.6856
